https://marshy-particle-825.notion.site/Code-Presentation-c64b8b3ae4484f348e9e73c1a0b3079c

# Code Presentation !

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import time

from PIL import Image
# import cv2

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms as tfs
torch.manual_seed(42)
# import albumentations

from glob import glob

from sklearn.model_selection import train_test_split

### save result ###
import hashlib
import json
from os import listdir
from os.path import isfile, join
import pandas as pd

### model argparse and start learn ###
import argparse
from copy import deepcopy
import multiprocessing as mp 
from multiprocessing import freeze_supp

### Import Data

In [ ]:
X = []
X.extend(glob('./data/train/*.png'))
X.extend(glob('./data/train/*.jpg'))
DATA_PATH_TRAIN_LIST, DATA_PATH_TEST_LIST = train_test_split(X, 
                                                            test_size=0.1, 
                                                            random_state=42)
# print(len(DATA_PATH_TRAIN_LIST))
# print(len(DATA_PATH_VAL_LIST))
# print(len(DATA_PATH_TEST_LIST))

### Image augmentation function

In [ ]:
class TrainImageTransform():
    
    def __init__(self):
        self.im_aug = tfs.Compose([
            tfs.Resize(256),
            tfs.CenterCrop(128),
            # tfs.RandomPerspective(), # 실험3. 입체방향 각도의 인식(param부터 다시 얻어와야할듯)
            # tfs.RandomRotation(360),
            tfs.RandomGrayscale(p=1),
            tfs.ColorJitter(brightness=0.5, contrast=0.5, hue=0.5),
            tfs.ToTensor(),
            tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])

    def __call__(self, img):
        degree = np.random.randint(359) 
        x = img.rotate(degree)
        x = self.im_aug(x)
        return x, degree


class TestImageTransform():
    
    def __init__(self):
        self.im_aug = tfs.Compose([
            tfs.Resize(256),
            tfs.CenterCrop(128),
            # tfs.RandomPerspective() # 실험3. 성능의 차이를 보자 !
            tfs.ToTensor(),
            tfs.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
            ])

    def __call__(self, img):
        degree = np.random.randint(359) # 실험2. iter에 다른 degree
        x = img.rotate(degree)
        x = self.im_aug(x)
        return x, degree

## 문제점

### 1. Torchvision 전처리 속도 문제

GPU

### 2. 로딩 준비



### Dataset function

In [ ]:
class Img_Dataset(Dataset):
    
    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path).convert('RGB') # 픽셀당 추가 알파 채널이 있다고 생각 하므로 채널이 3개가 아닌 4개입니다.
        img_transformed, angle = self.transform(img)

        return img_transformed, angle

In [ ]:
train_val_dataset = Img_Dataset(file_list=DATA_PATH_TRAIN_LIST,
                        transform=TrainImageTransform())

train_dataset, val_dataset = random_split(train_val_dataset, [9000, 1753])

test_dataset = Img_Dataset(file_list=DATA_PATH_TEST_LIST,
                        transform=TestImageTransform())

partition = {'train': train_dataset, 'val':val_dataset, 'test':test_dataset}

## VGGnet?

최근에는 이미지 특징(feature)을 추출하는 데 이용되는 등 기본 네트워크 모델로 활용되고 있으며, 매우 많은 메모리를 이용하여 연산한다는 단점이 있다.

[VGGNet - Organize everything I know documentation](https://oi.readthedocs.io/en/latest/computer_vision/cnn/vggnet.html)

[[논문 리뷰] VGGNet(2014) 리뷰와 파이토치 구현](https://deep-learning-study.tistory.com/398)

### VGGnet Model composition

In [ ]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
}

In [ ]:
class CNN(nn.Module):
    
    def __init__(self, model_code, in_channels, out_dim, act, use_bn):
        super(CNN, self).__init__()
        
        if act == 'relu':
            self.act = nn.ReLU()
        elif act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif act == 'tanh':
            self.act = nn.TanH()
        else:
            raise ValueError("Not a valid activation function code")
        
        self.layers = self._make_layers(model_code, in_channels, use_bn)
        self.classifer = nn.Sequential(nn.Linear(8192, 1024),
                                       self.act,
                                       nn.Linear(1024, out_dim))
        
    def forward(self, x):
        x = self.layers(x)
        x = x.view(x.size(0), -1)
        x = self.classifer(x)
        return x
        
    def _make_layers(self, model_code, in_channels, use_bn):
        layers = []
        for x in cfg[model_code]:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels=in_channels,
                                     out_channels=x,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)]
                if use_bn:
                    layers += [nn.BatchNorm2d(x)]
                layers += [self.act]
                in_channels = x
        return nn.Sequential(*layers)

### Train, Val

In [ ]:
def train(net, partition, optimizer, criterion, args):

    trainloader = DataLoader(partition['train'], 
                            batch_size=args.train_batch_size, 
                            shuffle=True, num_workers=args.num_workers)
    net.train()

    correct = 0
    total = 0
    train_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        optimizer.zero_grad() 

        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

def validate(net, partition, criterion, args):

    valloader = DataLoader(partition['val'], 
                            batch_size=args.test_batch_size, 
                            shuffle=False, num_workers=args.num_workers)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        if __name__ == '__main__':
            mp.freeze_support()
            for data in valloader:
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                outputs = net(images)

                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

### Test, Early stopping, Save model

In [ ]:
def test(net, partition, args):

    testloader = DataLoader(partition['test'], 
                            batch_size=args.test_batch_size, 
                            shuffle=False, num_workers=args.num_workers)
    net.eval()
    
    correct = 0
    total = 0
    current_labels = []
    current_preds = []
    with torch.no_grad():
        if __name__ == '__main__':
            mp.freeze_support()
            for data in testloader:
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()

                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                current_labels.extend(labels)
                current_preds.extend(predicted)

        test_acc = 100 * correct / total
    return test_acc, current_labels, current_preds

def experiment(partition, args):
    net = CNN(model_code = args.model_code,
            in_channels = args.in_channels,
            out_dim = args.out_dim,
            act = args.act,
            use_bn = args.use_bn)
    net.cuda()

    criterion = nn.CrossEntropyLoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        optimizer = optim.RMSprop(net.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    min_val_loss = np.Inf
    n_epochs_stop = 10
    epochs_no_improve = 0
    early_stop = False
    iter = 0

    for epoch in range(args.epoch):  # loop over the dataset multiple times
        ts = time.time()
        net, train_loss, train_acc = train(net, partition, optimizer, criterion, args) 
        val_loss, val_acc = validate(net, partition, criterion, args) 
        te = time.time()
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        print('Epoch {}, Acc(train/val): {:2.2f}/{:2.2f}, Loss(train/val) {:2.2f}/{:2.2f}. Took {:2.2f} sec'.format(epoch, train_acc, val_acc, train_loss, val_loss, te-ts))
        
        if val_loss < min_val_loss:
        # Save the model
            save_model_path = f'./{str(args.lr).split(".")[1]}-{args.model_code[3:]}-{args.epoch}.pt'
            torch.save(net.state_dict(), save_model_path) 
            epochs_no_improve = 0
            min_val_loss = val_loss

        else:
            epochs_no_improve += 1
        iter += 1
        if epoch > 9 and epochs_no_improve == n_epochs_stop:
            print('Early stopping!' )
            early_stop = True
            break
        else:
            continue

    test_acc, current_labels, current_preds = test(net, partition, args)
    
    result = {}
    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['train_accs'] = train_accs
    result['val_accs'] = val_accs
    result['train_acc'] = train_acc
    result['val_acc'] = val_acc
    result['test_acc'] = test_acc

    result['test_labels'] = current_labels
    result['test_preds'] = current_preds

    return vars(args), result

### save the learning results

In [ ]:
# !mkdir results

def save_exp_result(setting, result, args):
    exp_name = setting['exp_name']
    del setting['epoch']
    del setting['test_batch_size']

    hash_key = hashlib.sha1(str(setting).encode()).hexdigest()[:6]
    save_model_path = f'{str(args.lr).split(".")[1]}-{args.model_code[3:]}-{args.epoch}'
    filename = './results/{}-{}-{}.json'.format(exp_name, hash_key, save_model_path)
    result.update(setting)
    with open(filename, 'w') as f:
        json.dump(result, f)

    
def load_exp_result(exp_name):
    dir_path = './results'
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) if '.json' in f]
    list_result = []
    for filename in filenames:
        if exp_name in filename:
            with open(join(dir_path, filename), 'r') as infile:
                results = json.load(infile)
                list_result.append(results)
    df = pd.DataFrame(list_result) # .drop(columns=[])
    return df

### Handling

In [ ]:
# ====== Random seed Initialization ====== #
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.exp_name = "exp1_lr_model_code"

# ====== Model ====== #
args.model_code = 'VGG11'
args.in_channels = 3
args.out_dim = 360
args.act = 'relu'

# ====== Regularization ======= #
args.l2 = 0.00001
args.use_bn = True

# ====== Optimizer & Training ====== #
args.optim = 'Adam' #'RMSprop' #SGD, RMSprop, ADAM...
args.lr = 0.0001
args.epoch = 200

args.num_workers = 4
args.train_batch_size = 64
args.test_batch_size = 64

# ====== Experiment Variable ====== #
name_var1 = 'lr'
name_var2 = 'model_code'
list_var1 = [0.0001, 0.00001]
list_var2 = ['VGG13', 'VGG11', 'VGG16']

if __name__ == '__main__':
    mp.freeze_support()
    for var1 in list_var1:
        for var2 in list_var2:
            setattr(args, name_var1, var1)
            setattr(args, name_var2, var2)

            setting, result = experiment(partition, deepcopy(args))
            save_exp_result(setting, result)